In [1]:
import time
from datetime import datetime
import io
import sys
import os
import zipfile
import requests
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter, Retry

In [2]:
CONFIG = load_dotenv(".env")
url_after_2018 = os.getenv("URL_AFTER_2018")
attributes = os.getenv("ATTRIBUTES")
wktnw = os.getenv("WKTNW")
wktne = os.getenv("WKTNE")
wktsw = os.getenv("WKTSW")
wktse = os.getenv("WKTSE")
wktc = os.getenv("WKTC")
wktv = os.getenv("WKTV")
default_email = os.getenv("EMAIL")
api_key = os.getenv("API_KEY")
destination = os.getenv('DESTINATION')

In [4]:
start=2019
end=2019
interval=15
email= default_email
dest = destination
items = range(10)

start_year = 2019
end_year = 2019
download_url_list = []
print(download_url_list)


[]


In [5]:
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
}

In [7]:
wk_list = [wktc, wktne, wktnw, wktse, wktsw, wktv]
for i in wk_list:
    payload = "names={}&leap_day=false&interval={}&utc=false&attributes={}&email={}&wkt={}".format(
        start, interval, attributes, email, i)
    response = requests.request(
        "POST", url_after_2018, data=payload, headers=headers)
    time.sleep(60)
    jsonResponse = response.json()
    print(jsonResponse)
    # print(jsonResponse['outputs']['downloadUrl'])
    time.sleep(60)
    download_url_list.append(
        jsonResponse['outputs']['downloadUrl'])
print(download_url_list)


{'inputs': {'body': {'names': '2019', 'leap_day': 'false', 'interval': '15', 'utc': 'false', 'attributes': 'ghi,dni,wind_speed,air_temperature,relative_humidity', 'email': 'natalia.lopez17@upr.edu', 'wkt': 'POLYGON((-65.35 18.27,-65.35 18.35,-65.21 18.35,-65.21 18.27,-65.35 18.27))'}, 'params': {}, 'query': {}}, 'metadata': {'version': '2.0.0'}, 'status': 400, 'errors': ['Request size exceeds maximum limit for single request.', 'See https://developer.nrel.gov/docs/solar/nsrdb/guide/', 'Data processing failure.']}


In [ ]:
retry_strategy = Retry(total=3, status_forcelist=[404, 400, 405, 403, 429, 500, 502, 503, 504],
                       method_whitelist=["HEAD", "GET", "OPTIONS", "POST"], backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
print("fetching responses...")
for url in download_url_list:
    responses = http.get(url, timeout=10)
    print(f"timeout_http_adapter() responses = {responses}")
    time.sleep(60)

In [ ]:
for i in download_url_list:
    solar_res = requests.get(i)
    print(solar_res)
    with zipfile.ZipFile(io.BytesIO(solar_res.content)) as solar_zip_file:
        print("downloading zip file to destination....")
        solar_zip_file.extractall(dest)
